<a href="https://colab.research.google.com/github/hkdan502/NLP/blob/main/doc_cls_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=233aebecb5706266852c1a273348df8f8a2cd459435a7d7c7ad315aa1ae9e35a
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

- pretrained_model : 단 해당 모델은 허깅페이스에 등록되어 있어야 한다.
- downstream_model_dir : 앞 절에서 파인튜닝한 모델의 체크포인트 저장 위치(확장자가 ckpt인 파일이 하나 이상 있어야 한다.)
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무것도 입력하지 않으면 128(default)

In [3]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [9]:
args.downstream_model_checkpoint_fpath #앞에서 파인튜닝한 모델의 체크포인트

'/gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt'

In [10]:
args.pretrained_model_name

'beomi/kcbert-base'

In [5]:
from transformers import BertTokenizer #토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

In [4]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath, #앞에서 파인튜닝한 모델의 체크포인트 읽기
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained( #앞 절에서 파인튜닝 때 사용한 pretrained_model_name에 해당하는 모델의 설정값 읽기
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config) #BERT모델 초기화

#BERT모델에 체크포인트(fine_tuned_model_ckpt)를 주입!
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#모델이 평가모드로 전환. 드롭아웃 등 학습때만 사용한 기법들을 모두 무효화한다.
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [6]:
def inference_fn(sentence):
    inputs = tokenizer( #토큰화 수행 후, input_ids, attention_mask, token_type_ids 생성
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    print('*****[LOG] inputs: ', inputs)


    with torch.no_grad():
        #inputs를 pytorch tensor로 변환한 뒤 모델에 입력
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        #모델출력값(outputs.logits)은 소프트맥스 함수 적용 전의 logit형태인데, 
        prob = outputs.logits.softmax(dim=1)
        print('*****[LOG] prob: ', prob)
        #소프트맥스 함수로 모델 출력을 polarity prob.(소수점 4자리로 반올림)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        #예측 확률의 최댓값 위치에 따라 pred(positive/negative) 만들기
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        # 웹 페이지에서 긍/부정 막대 길이 조정
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [12]:
inference_fn('ㅇㅈ')

{'sentence': 'ㅇㅈ',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.7112',
 'negative_data': '부정 0.2888',
 'positive_width': '71.12%',
 'negative_width': '28.88%'}

In [13]:
inference_fn('아닌데')

{'sentence': '아닌데',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.385',
 'negative_data': '부정 0.615',
 'positive_width': '38.5%',
 'negative_width': '61.5%'}

In [14]:
inference_fn('글쎄')

{'sentence': '글쎄',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0061',
 'negative_data': '부정 0.9939',
 'positive_width': '0.61%',
 'negative_width': '99.39%'}

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [7]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시(w/ flask)
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [11]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 158, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 597, in urlopen
    httplib_response = self._make_request(conn, method, url,
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File 